# Definitions

In [ ]:
pip install qutip

In [ ]:
pip install cvxpy==1.1.17

In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
from qutip import piqs
import datetime
import math
import itertools as iter
import scipy.special
from scipy import linalg as la 
from numpy import linalg as lin 
import cvxpy as cp
from cvxpy import reductions as cpr

In [ ]:
def nCk (n,k):
  return math.factorial(n)/(math.factorial(k)*math.factorial(n-k))

def GnuInit (g,n,u):
  L0=[]
  for i in range(0,g*n*u+1):
    L0.append(0)
  L1=[]
  for i in range(0,g*n*u+1):
    L1.append(0)
  for i in range(0,n+1):
    if int(i/2)==i/2:
      L0[i*g]=(0.5**((n-1))*nCk(n,i))**0.5
    if int(i/2)!=i/2:
      L1[i*g]=(0.5**((n-1))*nCk(n,i))**0.5
  return L0,L1

In [ ]:
def globsym(N,p):
  jx=qutip.spin_Jx(N/2)
  jy=qutip.spin_Jy(N/2)
  jz=qutip.spin_Jz(N/2)
  kr=[]
  kr.append(np.sqrt(p)*jx)
  kr.append(np.sqrt(p)*jy)
  kr.append(np.sqrt(p)*jz)
  kr.append((1-(p*(N/2)*((N/2)+1)))**0.5*qeye(N+1))
  return kr

In [ ]:
def Acoff (vec,J,M):
  if vec == 0:
    return M
  if vec == 1:
    return ((J-M)*(J+M+1))**0.5
  if vec == -1:
    return ((J+M)*(J-M + 1))**0.5
def deg (J,N):
  return math.factorial(N)*(int(2*J+1))/(math.factorial(int(N/2 -J))*math.factorial(int(N/2 +J+1)))
def adeg (J,N):
  result=0
  for i in range(int(2*J),int(N+1)):
    result += deg(i/2,N)
  return result
def JM (J,M):
  a=np.zeros(int(2*J+1))
  if np.abs(M) > J:
    return Qobj(a)
  a[int(J+M)]=1
  b=Qobj(a)
  return b
def locsym(N,p):
  J=N/2
  M=J
  E=0*qeye(int(N+1))
  for i in range(0,int(2*J+1)):
    E+=p*Acoff(0,J,M)*(1+((adeg(J+1,2*J)*(2*J +1))/(deg(J,2*J)*(J+1))))**0.5/((2*J)**0.5)*JM(J,M)*JM(J,M).dag() + p*Acoff(-1,J,M)*(1+((adeg(J+1,2*J)*(2*J +1))/(deg(J,2*J)*(J+1))))**0.5/(2*J)**0.5*JM(J,M-1)*JM(J,M).dag() + p*Acoff(+1,J,M)*(1+((adeg(J+1,2*J)*(2*J +1))/(deg(J,2*J)*(J+1))))**0.5/(2*J)**0.5*JM(J,M+1)*JM(J,M).dag()
    M=M-1
  K=1*(1-3*p)*qeye(int(N+1))
  return [E,K]

In [ ]:
def BKR (u0,u1,kr,t,N):
  Pc=u0*u0.dag()+u1*u1.dag()
  rho=(1/(Pc.tr()))*Pc
  Arho=0*rho
  Crho=0*rho
  R=[]
  krl=[]
  F=[]
  Rvec=[]
  Fe=0
  PLHrho=[]
  ps=0.0000000001
  IF=[]
  Fuc=[]
  IFuc=[]
  Fue=0
  Rplh=[]
  rhovec=[]
  X=0*qeye((N+1)**2)
  rho=rho.full()
  Rk=0*qeye(N+1)
  Fe2=[]
  for i in range(0,N+1):
    for j in range(0,N+1):
      rhovec.append(rho[j][i])
  rhovec=Qobj([rhovec])
  rho=Qobj(rho)
  for i in range(0,len(t)):
    Rvec=[]
    Arho=0*rho
    krl=kr(N,t[i])
    for k in range(0,len(krl)):
      Arho+=krl[k]*rho*(krl[k]).dag()
    PLHrho=Arho.full()
    PLHrho[np.abs(PLHrho) < ps] = 0
    PLHrho=la.pinvh(PLHrho)
    Arho=Qobj(PLHrho)
    Crho=0*rho
    #for j in range(0,len(krl)):
      #R=(rho.sqrtm()*krl[j].dag()*Arho.sqrtm())
      #Crho+=R*Arho*R.dag()
    Fe=0
    Fue=0
    Rvec=[]
    R=[]
    for k in range(0,len(krl)):
      Rk=rho.sqrtm()*krl[k].dag()*Arho.sqrtm()
      Rplh=[]
      for i in range(0,N+1):
        for j in range(0,N+1):
          Rplh.append(Rk[j,i])
      R.append(Qobj([Rplh]))
    X=0*qeye((N+1)**2)
    for i in range(0,len(R)):
      for j in range(0,len(krl)):
        X+=(Qobj(tensor(qeye(N+1),krl[j].trans()).full())*R[i].dag())*(Qobj(tensor(qeye(N+1),krl[j].trans()).full())*R[i].dag()).dag()
    Fe2.append((rhovec*X*rhovec.dag())[0][0][0])
    for i in range(0,len(krl)):
      for j in range(0,len(krl)):
        Fe+=0.5*(np.abs((rho.sqrtm()*(krl[i]).dag()*u0*u0.dag()*Arho.sqrtm()*krl[j]*u0*u0.dag()).tr())**2)
        Fe+=0.5*(np.abs((rho.sqrtm()*(krl[i]).dag()*u1*u1.dag()*Arho.sqrtm()*krl[j]*u1*u1.dag()).tr())**2)
      Fue+=0.5*(np.abs((rho.sqrtm()*Qobj(la.pinvh(rho.full())).sqrtm()*krl[i]*u0*u0.dag()).tr())**2)+0.5*(np.abs((rho.sqrtm()*Qobj(la.pinvh(rho.full())).sqrtm()*krl[i]*u1*u1.dag()).tr())**2)
    F.append(Fe)
    Fuc.append(Fue)
  for i in range(0,len(F)):
    IF.append((1-F[i]))
    IFuc.append(1-Fuc[i])
  plot=plt.loglog(t,IF,label = 'Corrected')
  #plt.loglog(t,IFuc, label = 'Uncorrected')
  #plt.loglog(t,Fe2,label='alt calc')
  plt.legend()
  return IF

In [ ]:
def SDP (u0,u1,kr,t,N):
  rho= (u0*u0.dag()+u1*u1.dag())/2
  rho=rho.full()
  rhov=[]
  Fe=[]
  IFe=[]
  Fu=[]
  IFu=[]
  R=[]
  for i in range(0,N+1):
    for j in range(0,N+1):
      rhov.append(rho[j][i])
  rhov=Qobj([rhov])
  C=0*qeye((N+1)**2)
  krt=kr(N,0)
  id=(u0*u0.dag()+u1*u1.dag()).full()
  for k in range(0,len(t)):
    C=0*qeye((N+1)**2)
    print(k)
    krt=kr(N,t[k])
    for i in range(0,len(krt)):
      C+=Qobj(tensor(qeye(N+1),krt[i].conj()).full())*rhov.dag()*(Qobj(tensor(qeye(N+1),krt[i].conj()).full())*rhov.dag()).dag()
    C=np.real(C.full())
    X=cp.Variable(((N+1)**2,(N+1)**2),symmetric=True)#variables
    constraints = [X >> 0] #PSD matrix
    constraints+= [sum(X[l+(N+1)*j,l+(N+1)*k] for l in range(0,N+1))==id[j,k] for j in range(0,N+1) for k in range(0,N+1)] #ptrace gives X_2=I
    prob = cp.Problem(cp.Maximize(cp.trace(C @ X)),constraints)
    prob.solve(solver='SCS')
    Xmat=Qobj(X.value)
    C=Qobj(C)
    Fe.append((Xmat*C).tr())
    Fu.append((rhov*C*rhov.dag()).tr())
    R.append(Xmat)
  for i in range(0,len(t)):
    IFe.append((1-Fe[i]))
    IFu.append(1-Fu[i])
  plt.loglog(t,IFe,label='Corrected')
  #plt.loglog(t,IFu,label='Uncorrected')
  plt.legend()
  return IFe, R

In [ ]:
def lim_case(u0,u1,kr,t,N):
  krt=kr(N,0)
  ev=[]
  F=[]
  out0=u0
  out1=u1
  in0=0*u0*u0.dag()
  in1=0*u1*u1.dag()
  for i in range(0,len(t)):
    krt=kr(N,t[i])
    R=0*qeye((N+1)**2)
    in0=0*u0*u0.dag()
    in1=0*u1*u1.dag()
    for k in range(0,len(krt)):
      in0+=krt[k]*u0*u0.dag()*krt[k].dag()
      in1+=krt[k]*u1*u1.dag()*krt[k].dag()
    R=tensor(in0.trans(),out0*out0.dag())+tensor(in1.trans(),out0*out0.dag())
    ev=R.eigenenergies()
    F.append(1-ev[-1])
  plt.loglog(t,F)
  return F

# Code Guide

The three simulations are given as:

BKR (u0,u1,kr,t,N)

plt_lim_case(u0,u1,kr,t,N)

optERU (u0,u1,kr,t,N)

Where u0 and u1 are the logical codewords, kr is the Krauss operator for the noise model (choice between globsym and locsym for global symmetric noise and local symmetric noise respectively), t is an np.array of timesteps and N is the number of qubits in your system.



# Data from previous runs

In [ ]:
t=np.linspace(0.0001,0.01,10)

In [ ]:
bkrgross=[0.008075000000051347, 0.09690000000000376, 0.18572500000000125, 0.27455000000000185, 0.3633750000000009, 0.4521999999999984, 0.5410249999999988, 0.6298499999999994, 0.7186750000000003, 0.8075000000000007]
entgross=[0.008081984008515009, 0.09690491380639765, 0.18572972467814652, 0.2745499543721487, 0.36337490143481266, 0.45219997219177166, 0.5410250069434392, 0.6298499997783373, 0.7186747086346913, 0.7308334390113524]
limgross=[0.008075000000000387, 0.09689999999999999, 0.18572500000000014, 0.27454999999999985, 0.3633749999999998, 0.4521999999999998, 0.541025, 0.62985, 0.718675, 0.7308333333333331]
bkrgnu=[0.0072000000000413955, 0.08640000000000314, 0.16560000000000075, 0.24479999999999857, 0.3239999999999984, 0.40319999999999956, 0.4823999999999995, 0.5615999999999987, 0.6407999999999999, 0.7199999999999998]
entgnu=[0.007200240697589955, 0.08639986076577133, 0.16560004748177615, 0.2448093592128905, 0.3240032806395132, 0.403200105282162, 0.48239990842725045, 0.5616000163816468, 0.6408000145738698, 0.7200093921232473]
limgnu=[0.007199999999999873, 0.0863999999999997, 0.16559999999999975, 0.24479999999999946, 0.32399999999999984, 0.4032, 0.48239999999999994, 0.5615999999999999, 0.6407999999999998, 0.72]
bkrmin=[0.003244774293648778, 0.03896246109367785, 0.07470554482175573, 0.11047281571131384, 0.1462699867130891, 0.1821078759342145, 0.21800512029835029, 0.2539953341392618, 0.2901459522207459, 0.326619134506563]
entmin=[0.00807382285655489, 0.09690511431524329, 0.18575365036190616, 0.2745509158975705, 0.363370474503631, 0.4522038733789726, 0.5410250002664176, 0.5847865363413766, 0.5262451700803091, 0.4676856188987156]
limmin=[0.0031854030070119776, 0.03793864091585575, 0.0721194833389962, 0.10565160202532309, 0.13844732640292556, 0.17040619429573023, 0.20141348001815695, 0.23133874219830952, 0.26003437758413495, 0.2873339838883535]
bkrgkp=[0.0008500000000001284, 0.010199999999999987, 0.019549999999999734, 0.028899999999999593, 0.03824999999999956, 0.04759999999999964, 0.056949999999999834, 0.06630000000000014, 0.07564999999999988, 0.08499999999999941]
entgkp=[0.008073094753090593, 0.09689972777302058, 0.18572650593272444, 0.2745502156425208, 0.3633745747408994, 0.45220022667259574, 0.515922444283706, 0.4364502759818665, 0.3569750182078808, 0.27751597377709314]
limgkp=[0.0008500000000001284, 0.010199999999999987, 0.019550000000000067, 0.028899999999999926, 0.038249999999999784, 0.047599999999999865, 0.056949999999999834, 0.06630000000000003, 0.07565, 0.08499999999999996]

In [ ]:
bkrgrossL= [0.0005999099169857747, 0.007187039997958888, 0.01375239000057027, 0.020295959999466984, 0.026817750000054374, 0.033317760000052266, 0.03979598999991052, 0.04625243999999251, 0.05268711000002302, 0.05910000000004312]
bkrgkpL= [0.0005998316237122436, 0.007180919658958507, 0.013729907395672702, 0.020246829970580915, 0.026731687472039578, 0.033184479984807336, 0.03960520748735363, 0.045993869991553415, 0.05235046749323802, 0.058674999994648425]
bkrminL= [0.0005997960242534184, 0.007170596411897767, 0.013691982363750466, 0.020163953287385628, 0.02658650879472413, 0.032959648500381133, 0.03928337201886101, 0.04555767896250151, 0.05178256894529243, 0.05795804158002871]
bkrgnuL= [0.0005999100324642903, 0.00718703999934589, 0.013752390000065784, 0.020295960000515367, 0.026817750000081242, 0.033317759999826446, 0.039795990000139225, 0.04625244000006312, 0.05268710999996884, 0.0590999999999684]
limgrossL= [0.0005999099999999258, 0.007187040000000144, 0.013752389999999837, 0.020295960000000002, 0.026817749999999974, 0.033317759999999974, 0.039795990000000114, 0.04625244000000006, 0.052687109999999926, 0.05910000000000026]
limgkpL= [0.0005998674999995179, 0.00718091999112358, 0.013729907379411266, 0.02024682942031375, 0.026731685709330377, 0.033184475676738545, 0.039605198582181256, 0.045993853509118665, 0.05235043935913164, 0.05867495484608054]
limminL= [0.000599795514854895, 0.007170554075947244, 0.013691826495898396, 0.020163611022780636, 0.026585904759213297, 0.03295870362521647, 0.039282002320368936, 0.04555579428526102, 0.051780071662248295, 0.057954825255512654]
limgnuL= [0.0005999100000000368, 0.007187040000000033, 0.013752389999999726, 0.020295960000000002, 0.026817749999999863, 0.03331775999999986, 0.03979598999999989, 0.04625244000000017, 0.05268710999999948, 0.05909999999999993]
entgrossL= [0.0006084160825639007, 0.007194326011659591, 0.013757572768303472, 0.020298227913755573, 0.0268164489776479, 0.03331247791621639, 0.03978663212357536, 0.0462392874510994, 0.052670853647304394, 0.05908174256672538]
entgnuL= [0.000574776366741081, 0.0071872851317026365, 0.013752452665692916, 0.0202956968032737, 0.02681716550115365, 0.03331682337637909, 0.039794797490657285, 0.04625134074283843, 0.052687123577266015, 0.05910305888765244]
entgkpL= [0.000597460165526198, 0.007184922388878245, 0.013751796882263467, 0.020295928256540585, 0.026817755838847823, 0.033317778415536514, 0.03979602275112182, 0.04625247835139634, 0.05268713992426055, 0.05910000651034397]
entminL= [0.0005985751434467446, 0.007185829953042178, 0.013751407490487533, 0.020295316003917874, 0.026817557961348548, 0.03331812842202153, 0.039797013686437066, 0.04625419012060261, 0.05268962363113061, 0.05910326962415002]